In [1]:
# The contents of this files are based on the tensor flow data creation tutorial
# https://www.tensorflow.org/tutorials/load_data/tfrecord

import tensorflow as tf
from level.LevelReader import LevelReader
from pathlib import Path


In [2]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [3]:
def image_example(image_string, label):
  image_shape = tf.io.decode_jpeg(image_string).shape

  feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }

  return tf.train.Example(features=tf.train.Features(feature=feature))

In [ ]:
from GameManagement.GameConnection import GameConnection
import os
from util.Config import Config
from GameManagement.GameManager import GameManager
from util import ProgramArguments

parser = ProgramArguments.get_program_arguments()
config = parser.parse_args()
config.game_folder_path = os.path.normpath('../science_birds/{os}')

config = Config(config)
game_connection = GameConnection(conf = config)
game_manager = GameManager(conf = config, game_connection = game_connection)
game_manager.start_game()

data_dict = dict()

for level in sorted(Path("train/generated/").glob('*.xml')):
    level_reader = LevelReader(str(level))
    parsed_level = level_reader.parse_level(str(level), blocks = True, pigs = True, platform = True)
    parsed_level.filter_slingshot_platform()

    level_idx = level.name.strip('level-').strip('.xml')

    parsed_level.normalize()
    parsed_level.create_polygons()
    level_structures = parsed_level.separate_structures()

    structure_data = dict()
    for idx, structure in enumerate(level_structures):
        current_structure_data = parsed_level.calc_structure_meta_data(structure)
        if structure_data.pig_amount == 0:
            continue
        xml_file = level_reader.create_level_from_structure(structure, move_to_ground = True)
        structure_data[idx] = dict(
            meta_data = current_structure_data,
            file = xml_file
        )

    game_connection.startAi(start_level = level_idx, end_level = level_idx)

    structure_imgs = parsed_level.create_img(per_structure = True, dot_version = True)
    game_connection.wait_till_all_level_played()
    data = game_connection.get_data()

In [ ]:
record_file = 'images.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
  for filename, label in image_labels.items():
    image_string = open(filename, 'rb').read()
    tf_example = image_example(image_string, label)
    writer.write(tf_example.SerializeToString())